# CONFIGURATION GOOGLE DRIVE

In [ ]:
print("🔗 Connexion à Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive monté avec succès")

🔗 Connexion à Google Drive...
Mounted at /content/drive
✅ Google Drive monté avec succès


# INSTALLATION DES DÉPENDANCES

In [ ]:
print("\n📦 Installation des bibliothèques...")
print("⏳ Installation de LightFM en cours...")
!pip install -q lightfm

print("⏳ Installation des dépendances supplémentaires...")
!pip install -q tqdm

import sys
print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor}")
print("✅ Toutes les dépendances installées")


📦 Installation des bibliothèques...
⏳ Installation de LightFM en cours...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for lightfm
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (lightfm)
⏳ Installation des dépendances supplémentaires...
✅ Python 3.12
✅ Toutes les dépendances installées


# CONFIGURATION DES CHEMINS

In [ ]:
import os
from pathlib import Path

print("\n📁 Configuration des chemins de travail...")

# Chemins principaux
BASE_PATH = "/content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys"
DATA_PATH = f"{BASE_PATH}/data"
OUTPUTS_PATH = f"{BASE_PATH}/outputs"

# Création de la structure de dossiers
directories_to_create = [
    f"{OUTPUTS_PATH}/models",
    f"{OUTPUTS_PATH}/figures",
    f"{OUTPUTS_PATH}/metrics",
    f"{OUTPUTS_PATH}/samples"
]

for directory in directories_to_create:
    os.makedirs(directory, exist_ok=True)

print(f"📂 Répertoire de base : {BASE_PATH}")
print(f"📂 Données : {DATA_PATH}")
print(f"📂 Résultats : {OUTPUTS_PATH}")
print("✅ Structure de dossiers créée")


📁 Configuration des chemins de travail...
📂 Répertoire de base : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys
📂 Données : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/data
📂 Résultats : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/outputs
✅ Structure de dossiers créée


# CHARGEMENT DES DATASETS H&M

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

print("\n📊 Chargement des datasets H&M...")

def charger_donnees_h2m():
    """
    Charge les trois datasets principaux H&M

    Returns:
        tuple: (df_articles, df_customers, df_transactions)
    """
    try:
        print("⏳ Chargement articles.csv...")
        df_articles = pd.read_csv(f"{DATA_PATH}/articles.csv")

        print("⏳ Chargement customers.csv...")
        df_customers = pd.read_csv(f"{DATA_PATH}/customers.csv")

        print("⏳ Chargement transactions_train.csv...")
        df_transactions = pd.read_csv(f"{DATA_PATH}/transactions_train.csv")

        return df_articles, df_customers, df_transactions

    except FileNotFoundError as e:
        print(f"❌ Erreur : Fichier manquant - {e}")
        print("📋 Vérifiez que les fichiers suivants sont présents dans /data/ :")
        print("   • articles.csv")
        print("   • customers.csv")
        print("   • transactions_train.csv")
        return None, None, None

    except Exception as e:
        print(f"❌ Erreur lors du chargement : {e}")
        return None, None, None

# Chargement des données
df_articles, df_customers, df_transactions = charger_donnees_h2m()

if df_articles is not None:
    print("✅ Tous les datasets chargés avec succès!")
else:
    print("❌ Échec du chargement des données")


📊 Chargement des datasets H&M...
⏳ Chargement articles.csv...
⏳ Chargement customers.csv...
⏳ Chargement transactions_train.csv...
✅ Tous les datasets chargés avec succès!


# VALIDATION DES DONNÉES

In [ ]:
if df_articles is not None:
    print("\n🔍 Validation et aperçu des datasets...")

    # Informations sur les dimensions
    print("\n📊 DIMENSIONS DES DATASETS")
    print("="*50)
    print(f"Articles     : {df_articles.shape[0]:>8,} lignes × {df_articles.shape[1]:>2} colonnes")
    print(f"Customers    : {df_customers.shape[0]:>8,} lignes × {df_customers.shape[1]:>2} colonnes")
    print(f"Transactions : {df_transactions.shape[0]:>8,} lignes × {df_transactions.shape[1]:>2} colonnes")

    # Colonnes disponibles
    print("\n🏷️ COLONNES DISPONIBLES")
    print("="*50)
    print(f"Articles    : {list(df_articles.columns)}")
    print(f"Customers   : {list(df_customers.columns)}")
    print(f"Transactions: {list(df_transactions.columns)}")

    # Statistiques rapides
    n_clients_uniques = df_transactions['customer_id'].nunique()
    n_articles_uniques = df_transactions['article_id'].nunique()
    n_transactions = len(df_transactions)

    print("\n📈 STATISTIQUES RAPIDES")
    print("="*50)
    print(f"Clients uniques      : {n_clients_uniques:>10,}")
    print(f"Articles uniques     : {n_articles_uniques:>10,}")
    print(f"Transactions totales : {n_transactions:>10,}")
    print(f"Sparsité estimée     : {(1 - n_transactions/(n_clients_uniques * n_articles_uniques))*100:>9.2f}%")

    # Aperçu des données
    print("\n👀 APERÇU DES PREMIÈRES LIGNES")
    print("="*50)
    print("\n📋 ARTICLES (3 premières lignes):")
    print(df_articles.head(3).to_string())

    print("\n👥 CUSTOMERS (3 premières lignes):")
    print(df_customers.head(3).to_string())

    print("\n🛒 TRANSACTIONS (3 premières lignes):")
    print(df_transactions.head(3).to_string())


🔍 Validation et aperçu des datasets...

📊 DIMENSIONS DES DATASETS
Articles     :  105,542 lignes × 25 colonnes
Customers    : 1,371,980 lignes ×  7 colonnes
Transactions : 31,788,324 lignes ×  5 colonnes

🏷️ COLONNES DISPONIBLES
Articles    : ['article_id', 'product_code', 'prod_name', 'product_type_no', 'product_type_name', 'product_group_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code', 'colour_group_name', 'perceived_colour_value_id', 'perceived_colour_value_name', 'perceived_colour_master_id', 'perceived_colour_master_name', 'department_no', 'department_name', 'index_code', 'index_name', 'index_group_no', 'index_group_name', 'section_no', 'section_name', 'garment_group_no', 'garment_group_name', 'detail_desc']
Customers   : ['customer_id', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code']
Transactions: ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']

📈 STATISTIQUES RAPIDES
Clients uniques    

# SAUVEGARDE POUR LES NOTEBOOKS SUIVANTS

In [ ]:
if df_articles is not None:
    print("\n💾 Sauvegarde des datasets pour les notebooks suivants...")

    import pickle
    from datetime import datetime

    # Préparation des données à sauvegarder
    datasets_complets = {
        'articles': df_articles,
        'customers': df_customers,
        'transactions': df_transactions,
        'metadata': {
            'date_creation': datetime.now().isoformat(),
            'n_articles': len(df_articles),
            'n_customers': len(df_customers),
            'n_transactions': len(df_transactions),
            'colonnes_articles': list(df_articles.columns),
            'colonnes_customers': list(df_customers.columns),
            'colonnes_transactions': list(df_transactions.columns)
        }
    }

    # Sauvegarde
    fichier_sauvegarde = f"{OUTPUTS_PATH}/datasets_complets.pkl"
    with open(fichier_sauvegarde, "wb") as f:
        pickle.dump(datasets_complets, f)

    print(f"✅ Datasets sauvegardés dans : {fichier_sauvegarde}")
    print(f"📊 Taille du fichier : {os.path.getsize(fichier_sauvegarde) / (1024*1024):.1f} MB")



💾 Sauvegarde des datasets pour les notebooks suivants...
✅ Datasets sauvegardés dans : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/outputs/datasets_complets.pkl
📊 Taille du fichier : 1765.4 MB


# RÉCAPITULATIF ET PROCHAINES ÉTAPES

In [ ]:
print("\n" + "="*70)
print("🎯 RÉCAPITULATIF DE LA CONFIGURATION")
print("="*70)

if df_articles is not None:
    print("✅ Configuration Google Drive")
    print("✅ Installation des dépendances")
    print("✅ Création de la structure de dossiers")
    print("✅ Chargement des datasets H&M")
    print("✅ Validation des données")
    print("✅ Sauvegarde pour notebooks suivants")

    print("\n🚀 PROCHAINES ÉTAPES")
    print("-"*30)
    print("1️⃣  01_Exploration_Donnees.ipynb")
    print("2️⃣  02_Echantillonnage_Donnees.ipynb")
    print("3️⃣  03_Preparation_Donnees.ipynb")
    print("4️⃣  04_Modele_Collaboratif.ipynb")
    print("5️⃣  05_Optimisation_Hyperparametres.ipynb")
    print("6️⃣  06_Evaluation_Modele.ipynb")
    print("7️⃣  07_Modele_Hybride.ipynb")
    print("8️⃣  08_Pipeline_Final.ipynb")

else:
    print("❌ Configuration incomplète")
    print("📋 Vérifiez la présence des fichiers de données")

print("\n🎉 Configuration terminée !")
print("="*70)


🎯 RÉCAPITULATIF DE LA CONFIGURATION
✅ Configuration Google Drive
✅ Installation des dépendances
✅ Création de la structure de dossiers
✅ Chargement des datasets H&M
✅ Validation des données
✅ Sauvegarde pour notebooks suivants

🚀 PROCHAINES ÉTAPES
------------------------------
1️⃣  01_Exploration_Donnees.ipynb
2️⃣  02_Echantillonnage_Donnees.ipynb
3️⃣  03_Preparation_Donnees.ipynb
4️⃣  04_Modele_Collaboratif.ipynb
5️⃣  05_Optimisation_Hyperparametres.ipynb
6️⃣  06_Evaluation_Modele.ipynb
7️⃣  07_Modele_Hybride.ipynb
8️⃣  08_Pipeline_Final.ipynb

🎉 Configuration terminée !
